<h4>8. Load the MNIST data (introduced in Chapter 3), and split it into a training set, a
validation set, and a test set (e.g., use 40,000 instances for training, 10,000 for validation, and 10,000 for testing). Then train various classifiers, such as a Random
Forest classifier, an Extra-Trees classifier, and an SVM. Next, try to combine
them into an ensemble that outperforms them all on the validation set, using a
soft or hard voting classifier. Once you have found one, try it on the test set. How
much better does it perform compared to the individual classifiers?</h4>

In [21]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split    

digits_data = fetch_openml('mnist_784', version=1, cache=True, as_frame=False)
X_train, X_test, y_train, y_test = train_test_split(digits_data.data, digits_data.target, test_size=0.2, random_state=42) 
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)  

c:\Users\cozie\Documents\Learning\Hands on ML\venv\lib\site-packages\sklearn\datasets\_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [22]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC

rnd_clf = RandomForestClassifier(random_state=42) 
ext_clf = ExtraTreesClassifier(random_state=42)           
svm_clf = LinearSVC(random_state=42)

predictors = [rnd_clf, ext_clf, svm_clf]    

for predictor in predictors:
    predictor.fit(X_train, y_train)

c:\Users\cozie\Documents\Learning\Hands on ML\venv\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\cozie\Documents\Learning\Hands on ML\venv\lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [23]:
[predictor.score(X_val, y_val) for predictor in predictors]     

[0.96625, 0.9716071428571429, 0.8520535714285714]

In [24]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
estimators = [('random_forest', rnd_clf), ('extra_trees', ext_clf), ('svm', svm_clf)]
ensemble_clf = VotingClassifier(estimators, voting='hard')
ensemble_clf.fit(X_train, y_train)

c:\Users\cozie\Documents\Learning\Hands on ML\venv\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\cozie\Documents\Learning\Hands on ML\venv\lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


VotingClassifier(estimators=[('random_forest',
                              RandomForestClassifier(random_state=42)),
                             ('extra_trees',
                              ExtraTreesClassifier(random_state=42)),
                             ('svm', LinearSVC(random_state=42))])

In [25]:
ensemble_clf.score(X_val, y_val)    

0.9663392857142857

The result of using an ensemble model is worst, this could be due to the low performance of the SVM

In [26]:
del ensemble_clf.estimators_[2]

In [27]:
ensemble_clf.score(X_val, y_val)    

0.9686607142857143

The result without the SVM is better, however the ensemble model performs worst than the ET model

In [28]:
ensemble_clf.voting = 'soft'
ensemble_clf.score(X_val, y_val)

0.9699107142857143

Let's test the ensemble model with the test set and the individual models

In [33]:
print('Result of ensemble classifier on test set: ', ensemble_clf.score(X_test, y_test))
[estimator.score(X_test, y_test) for estimator in predictors]

Result of ensemble classifier on test set:  0.9677857142857142


[0.9651428571428572, 0.9678571428571429, 0.8565]

<h4> 9. Run the individual classifiers from the previous exercise to make predictions on
the validation set, and create a new training set with the resulting predictions:
each training instance is a vector containing the set of predictions from all your
classifiers for an image, and the target is the image’s class. Congratulations, you
have just trained a blender, and together with the classifiers they form a stacking
ensemble! Now let’s evaluate the ensemble on the test set. For each image in the
test set, make predictions with all your classifiers, then feed the predictions to the
blender to get the ensemble’s predictions. How does it compare to the voting classifier you trained earlier?</h4>


In [48]:
import numpy as np
X_val_pred = np.vstack((rnd_clf.predict(X_val), ext_clf.predict(X_val), svm_clf.predict(X_val))).T

In [49]:
rnd_forest_blender = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)
rnd_forest_blender.fit(X_val_pred, y_val)

RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)

In [50]:
rnd_forest_blender.oob_score_

0.9694642857142857

In [51]:
X_test_pred = np.vstack((rnd_clf.predict(X_test), ext_clf.predict(X_test), svm_clf.predict(X_test))).T
rnd_forest_blender.score(X_test_pred, y_test)

0.9663571428571428

This stacking ensemble performs worse than the voting classifier.